In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from collections import Counter

##############################
# Definition of Env variable # 
##############################
#Definition of the current env
# JUPYTER or G_COLAB
CURRENT_ENV = "G_COLAB"

#Use on Jupyter_env only 
DATA_DIR = "../../dev_GenJavaBdd/GenerationTest/"

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lecture des fichiers
get the number of sbt/bdd function

In [34]:
if CURRENT_ENV == "JUPYTER":
    list_folder = os.listdir(DATA_DIR)
    num_folder = len(list_folder)-2
elif CURRENT_ENV == "G_COLAB":
    with open("/content/drive/MyDrive/DATASET/sbt_bdd_Rubens/bdd.txt","r") as f:
        num_folder = len(f.readlines())
    with open("/content/drive/MyDrive/DATASET/sbt_bdd_Rubens/sbt.txt","r") as f:
        if (len(f.readlines()) != num_folder):
            error("[ERROR] number of BDD function does not match BDD number")
print("Number of function/Foler = ", num_folder)  # This would give length of files.


Number of function/Foler =  5000


# Creation des listes brutes SBT & BDD

In [35]:
#!pip install tqdm
from tqdm import tqdm
from time import sleep

In [36]:
bdd = []
sbt = []

#Create the list and save it as file
if CURRENT_ENV == "JUPYTER":
    print("Get the bdd and sbt list via folder")
    
    # Sleep to prevent fail tqdm print
    sleep(1)
    for i  in tqdm (range(num_folder), desc="loading..."):
    #for i  in range(num_folder):
        path_iteration = DATA_DIR+'/Dir_Fonction_'+str(i+1)
        bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
        stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
        bdd_txt = open(bdd_path,'r').read()
        sbt_txt = open(stb_path,'r').read()
        clean_bdd = bdd_txt.replace('\n', ' ')
        bdd.append(clean_bdd)
        sbt.append(sbt_txt)
    sleep(1)
    
    print("Save the BDD & SBT list as file")
    # Save the list as file
    #os.remove("bddi.txt")
    f_bdd = open("bdd.txt", "w+")
    f_bdd.write('\n'.join(bdd))
    f_bdd.close()

    #os.remove("sbt.txt")
    f_sbt = open("sbt.txt", "w+")
    f_sbt.write('\n'.join(sbt))
    f_sbt.close()
    
#Create the list from the txt file
else:
    print("Get the bdd and sbt list via the bdd.txt & sbt.txt")
    with open("/content/drive/MyDrive/DATASET/sbt_bdd_Rubens/bdd.txt") as f:
        tmp_bdd = f.read()
    bdd = tmp_bdd.split("\n")

    with open("/content/drive/MyDrive/DATASET/sbt_bdd_Rubens/sbt.txt") as f:
        tmp_sbt = f.read()
    sbt = tmp_sbt.split("\n")


Get the bdd and sbt list via the bdd.txt & sbt.txt


In [37]:
# Cette fonction sera utilisée pour récupérer les codes sbt et bdd dans un autre code python
def return_bdd_sbt():
    return bdd,sbt

# Recuperation des valeurs max de bdd et sbt

In [38]:
def return_max_len( text ):
    all_len = []
    for unique_code in text:
        code_split = unique_code.split(' ')
        all_len.append(len(code_split))
    return max(all_len)

# ######################################################
# Creation of the dict variale that replace SBT & BDD 's

In [39]:
'''
Create a nested dico that list all sbt variable and associeted itn with a TAG p_TAG + an number
:param p_sbt: list of sbt function. Each ligne of the list correspond to one sbt function
:param p_TAG: Tag associeted to the variable

return a nested dict with unique keys/value per dico
e.g.:
    m_dict[0][variable_name]
        return the associeted tag of the variable variable_name in the sbt function 0
'''
def create_dico_for_variable(p_sbt, p_TAG = "TG"):

    # Copy sbt in l_sbt which will be modify and returned
    l_sbt = p_sbt[:]
    # Stor the set use to create the dict
    curr_set = set()
    # Store the dict 
    curr_dic = {}
    # Tag use to remplace the variable in SBT
    SBT_var_tag = p_TAG

    # Get all variable name from SBT and stor it in the set
    # The variable from the SBT are found after the Key word "SimpleName"
    for i in range(len(l_sbt)):
        
        # Get the set of variable found with SimpleName keyword in the SBT
        l_start_w_var = l_sbt[i].split("SimpleName ")
        for j in range(1, len(l_start_w_var)):
            curr_variable = l_start_w_var[j].split(' ')[0]
            curr_set.add(curr_variable)
            
        # create the associeted dict 
        curr_dic[i] = {}
        for k in range(len(curr_set)):
            curr_dic[i][str(list(curr_set)[k])] = SBT_var_tag + str(k)
            l_sbt[i] = l_sbt[i].replace(' ' + str(list(curr_set)[k]) + ' ', ' ' + SBT_var_tag + str(k) + ' ')
            
        # reset the current_set
        curr_set = set()
    return curr_dic, l_sbt


Encode the SBT & BDD with the new variable using the dict

In [40]:
# --------------------------------------
# Encode the sbt & bdd with the dict var
# --------------------------------------

# Get the dict + sbt modified
m_dict, sbt_var_encoded = create_dico_for_variable(sbt)

# Modified the sbt
bdd_var_encoded = bdd[:]
for i in range(len(bdd)):
    for keys,values in m_dict[i].items():
        if len(keys) < 5:
            bdd_var_encoded[i] = bdd_var_encoded[i].replace(' '+ keys + ' ', ' '+ values + ' ')
        else:
            bdd_var_encoded[i] = bdd_var_encoded[i].replace(keys,values)


# Test with technich dico variable
# ###########################

# Encode SBT & BDD 
### => Each word corresponding to one idx

In [41]:
x = len(bdd_var_encoded[0].split(' '))
for i in range(1, len(bdd_var_encoded)):
  if x != len(bdd_var_encoded[48].split(' ')):
    print("HELLO")

In [42]:
bdd_var_encoded[0]

'Feature: Check the TG2     # Check the result to True   Scenario: bdd_TG2_1     Given TG4 = -9 and TG3 = 13     When TG0 call TG2     Then the result should be True    # Check the result to False   Scenario: bdd_TG2_2     Given TG4 = 0 and TG3 = 5     When TG0 call TG2     Then the result should be False '

In [43]:
set_of_words_sbt = set((' '.join(sbt_var_encoded)).split(' '))
set_of_words_bdd = set((' '.join(bdd_var_encoded)).split(' '))



set_of_words_bdd.add('sostoken ')
set_of_words_bdd.add(' eostoken')
set_of_words_bdd.add(' ')
set_of_words_sbt.add(' ')

word_to_ind_sbt = {word : ind for ind,word in enumerate(set_of_words_sbt)}
ind_to_word_sbt = {ind : word for ind,word in enumerate(set_of_words_sbt)}

word_to_ind_bdd = {word : ind for ind,word in enumerate(set_of_words_bdd)}
ind_to_word_bdd = {ind : word for ind,word in enumerate(set_of_words_bdd)}

number_folder = len(bdd_var_encoded)
max_words_sbt = return_max_len(sbt_var_encoded)+1
max_words_bdd = return_max_len(bdd_var_encoded)+1+8
number_of_words_sbt = len(set_of_words_sbt)
number_of_words_bdd = len(set_of_words_bdd)
#'''
print(' number of forlder :',number_folder)
print(' number maximum of word in one sbt code :',max_words_sbt)
print(' number maximum of word in one bdd code :',max_words_bdd)
print(' number of different word in sbt :',number_of_words_sbt)
print(' number of different word in bdd :',number_of_words_bdd)
#'''

print ("ind_to_word_bdd = ", ind_to_word_bdd)


 number of forlder : 5000
 number maximum of word in one sbt code : 256
 number maximum of word in one bdd code : 101
 number of different word in sbt : 43
 number of different word in bdd : 98
ind_to_word_bdd =  {0: '', 1: '15', 2: '18', 3: '13', 4: ' ', 5: 'the', 6: 'TG3', 7: '-11', 8: '8', 9: '14', 10: '-25', 11: 'to', 12: '-12', 13: 'and', 14: ' eostoken', 15: '-29', 16: '-27', 17: '2', 18: 'True', 19: '11', 20: 'bdd_TG2_2', 21: '-3', 22: '7', 23: '-15', 24: 'TG0', 25: '10', 26: 'sostoken ', 27: 'bdd_TG0_1', 28: 'bdd_TG5_2', 29: '27', 30: 'i', 31: 'bdd_TG3_1', 32: '12', 33: 'bdd_TG4_2', 34: '-6', 35: '25', 36: 'TG1', 37: 'result', 38: '-2', 39: 'Check', 40: '-23', 41: '-28', 42: '-7', 43: '-24', 44: '-17', 45: '-30', 46: 'False', 47: '-5', 48: '22', 49: 'bdd_TG2_1', 50: '-13', 51: '-19', 52: 'TG4', 53: '20', 54: 'bdd_TG3_2', 55: '9', 56: '-9', 57: '16', 58: '4', 59: '24', 60: '-10', 61: 'When', 62: 'be', 63: '-20', 64: '-26', 65: 'should', 66: 'bdd_TG0_2', 67: 'call', 68: '0', 69: 

# Creation of the Data Frame

In [44]:
df = pd.DataFrame(bdd_var_encoded, columns=['bdd'])
df['sbt'] = sbt_var_encoded
df['bdd_input'] = bdd_var_encoded
df['bdd_label'] = bdd_var_encoded
df.tail(5)

,bdd,sbt,bdd_input,bdd_label
4995,Feature: Check the TG2 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG2 # Check the result ...,Feature: Check the TG2 # Check the result ...
4996,Feature: Check the TG3 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG3 # Check the result ...,Feature: Check the TG3 # Check the result ...
4997,Feature: Check the TG3 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG3 # Check the result ...,Feature: Check the TG3 # Check the result ...
4998,Feature: Check the TG1 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG1 # Check the result ...,Feature: Check the TG1 # Check the result ...
4999,Feature: Check the TG3 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG3 # Check the result ...,Feature: Check the TG3 # Check the result ...


# Shuffle the arrays

In [45]:
encoder_input = np.array(df.sbt)
decoder_input = np.array(df.bdd_input)
decoder_label = np.array(df.bdd_label)


indices = np.arange(num_folder)
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_label = decoder_label[indices]

df.head(10)

,bdd,sbt,bdd_input,bdd_label
0,Feature: Check the TG2 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG2 # Check the result ...,Feature: Check the TG2 # Check the result ...
1,Feature: Check the TG0 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG0 # Check the result ...,Feature: Check the TG0 # Check the result ...
2,Feature: Check the TG3 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG3 # Check the result ...,Feature: Check the TG3 # Check the result ...
3,Feature: Check the TG2 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG2 # Check the result ...,Feature: Check the TG2 # Check the result ...
4,Feature: Check the TG4 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG4 # Check the result ...,Feature: Check the TG4 # Check the result ...
5,Feature: Check the TG0 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG0 # Check the result ...,Feature: Check the TG0 # Check the result ...
6,Feature: Check the TG2 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG2 # Check the result ...,Feature: Check the TG2 # Check the result ...
7,Feature: Check the TG4 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG4 # Check the result ...,Feature: Check the TG4 # Check the result ...
8,Feature: Check the TG1 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG1 # Check the result ...,Feature: Check the TG1 # Check the result ...
9,Feature: Check the TG0 # Check the result ...,( MethodDeclaration ( Modifier public ) Modi...,Feature: Check the TG0 # Check the result ...,Feature: Check the TG0 # Check the result ...


# Split SBT & BDD with encoder/decoder/label + format

In [46]:
total = num_folder
test_size = 0.1

train_encoder_input = encoder_input[:-int(total*test_size)]
train_decoder_input = decoder_input[:-int(total*test_size)]
train_decoder_label = decoder_label[:-int(total*test_size)]

test_encoder_input = encoder_input[-int(total*test_size):]
test_decoder_input = decoder_input[-int(total*test_size):]
test_decoder_label = decoder_label[-int(total*test_size):]

print("train dataset shape")
print(train_encoder_input.shape)
print(train_decoder_input.shape)
print(train_decoder_label.shape)

print("\n\ntest dataset shape")
print(test_encoder_input.shape)
print(test_decoder_input.shape)
print(test_decoder_label.shape)


train dataset shape
(4500,)
(4500,)
(4500,)


test dataset shape
(500,)
(500,)
(500,)


Il faut maintenant convertir ce code en sequences, puis padded ces sequences, on va pas ajouter les tokens de debut et fin maintenant, on les ajoutera apres avoir encodé

In [47]:
def encode(text,word_to_ind, dim1, dim2,type):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix = np.full((dim1,dim2),word_to_ind[' '])
    #print('Encoding Matrix shape : ',encoded_matrix.shape)
    for i,words in enumerate(text):
        j=0
        #print(vars)
        split_word = words.split(' ')
        if type == 1:
            encoded_matrix[i,j]=word_to_ind['sostoken ']
            j=j+1
        
        for word in (split_word):
            encoded_matrix[i,j]=word_to_ind[word]

            j=j+1
        if type == 2:
            encoded_matrix[i,j]=word_to_ind[' eostoken']
            j=j+1

    return encoded_matrix

In [48]:
def encode_unique(text):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix_unique = np.full(256,word_to_ind_sbt[' '])

    split_word = text.split(' ')

    for i,word in enumerate(split_word):
        encoded_matrix_unique[i]=word_to_ind_sbt[word]

    return encoded_matrix_unique

In [49]:
print("train dataset shape")
train_encoder_input_encoded = encode(train_encoder_input, word_to_ind_sbt,len(train_encoder_input),max_words_sbt,0)
train_decoder_input_encoded = encode(train_decoder_input, word_to_ind_bdd,len(train_decoder_input),max_words_bdd,1)
train_decoder_label_encoded = encode(train_decoder_label, word_to_ind_bdd,len(train_decoder_label),max_words_bdd,2)
print("\n\ntest dataset shape")

test_encoder_input_encoded = encode(test_encoder_input, word_to_ind_sbt,len(test_encoder_input),max_words_sbt,0)
test_decoder_input_encoded = encode(test_decoder_input, word_to_ind_bdd,len(test_decoder_input),max_words_bdd,1)
test_decoder_label_encoded = encode(test_decoder_label, word_to_ind_bdd,len(test_decoder_label),max_words_bdd,2)

train dataset shape


test dataset shape


In [50]:
test_encoder_input_encoded.shape

(500, 256)

In [51]:
def decode ( encoded_sequence, ind_to_char):
    decoded_sequence = []
    for var in encoded_sequence:
        decoded_sequence.append(ind_to_char[var])
    return ' '.join(decoded_sequence)

In [52]:
# TEST
decode(train_decoder_label_encoded[0],ind_to_word_bdd)

'Feature: Check the TG4     # Check the result to True   Scenario: bdd_TG4_1     Given TG3 = -10 and TG1 = -4     When i call TG4     Then the result should be True    # Check the result to False   Scenario: bdd_TG4_2     Given TG3 = -9 and TG1 = -4     When i call TG4     Then the result should be False   eostoken                '

# Creation du model

In [53]:
number_of_words_bdd

98

In [54]:
from keras.layers import Input,Embedding,LSTM,Dense,Concatenate,Attention
from keras.models import Model
#from keras.utils import plot_model
from keras import backend as K

#hyperparameters
embedding_size = 256
hidden_size = 256

# trainer model (generator model will use the same encoder tho)
encoder_input = Input(shape=[max_words_sbt]) # size chain SBT 
encoder_embedding = Embedding(max_words_sbt,embedding_size,mask_zero=True)
encoder_embedded = encoder_embedding(encoder_input)

encoder_lstm1 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output1,encoder_h1,encoder_c1 = encoder_lstm1(encoder_embedded)

encoder_lstm2 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output2,encoder_h2,encoder_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output3,encoder_h3,encoder_c3 = encoder_lstm3(encoder_output1)

decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(max_words_bdd,embedding_size,mask_zero=True) # len chain bdd
decoder_embedded = decoder_embedding(decoder_input)

decoder_lstm = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
decoder_output,_,_ = decoder_lstm(decoder_embedded,initial_state=[encoder_h3,encoder_c3])

attn_layer = Attention()
attn_context = attn_layer([decoder_output,encoder_output3])

decoder_output = Concatenate(axis=-1)([decoder_output,attn_context])
tanh_dense= Dense(hidden_size*2,activation=K.tanh)
decoder_output = tanh_dense(decoder_output)

softmax_dense = Dense(number_of_words_bdd,activation='softmax')
decoder_output = softmax_dense(decoder_output)

trainer_model = Model([encoder_input,decoder_input],decoder_output)
trainer_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [55]:
trainer_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 256, 256)     65536       input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 256, 256), ( 525312      embedding_2[0][0]                
____________________________________________________________________________________________

In [56]:
print(len(train_decoder_label_encoded))
print(len(train_encoder_input_encoded))
print(len(train_decoder_input_encoded))

4500
4500
4500


In [57]:
print(train_decoder_label_encoded.shape)
print(train_encoder_input_encoded.shape)
print(train_decoder_input_encoded.shape)

(4500, 101)
(4500, 256)
(4500, 101)


In [58]:
trainer_hist =trainer_model.fit([train_encoder_input_encoded,train_decoder_input_encoded],train_decoder_label_encoded,epochs=10,batch_size=128,validation_split=0.2)

Epoch 1/10
29/29 [==============================] - 129s 4s/step - loss: 2.3120 - accuracy: 0.2304 - val_loss: 1.5100 - val_accuracy: 0.3538
Epoch 2/10
29/29 [==============================] - 112s 4s/step - loss: 1.3303 - accuracy: 0.4370 - val_loss: 0.8088 - val_accuracy: 0.7206
Epoch 3/10
29/29 [==============================] - 112s 4s/step - loss: 0.6924 - accuracy: 0.7677 - val_loss: 0.4044 - val_accuracy: 0.8166
Epoch 4/10
29/29 [==============================] - 111s 4s/step - loss: 0.3756 - accuracy: 0.8224 - val_loss: 0.3191 - val_accuracy: 0.8263
Epoch 5/10
29/29 [==============================] - 112s 4s/step - loss: 0.3178 - accuracy: 0.8273 - val_loss: 0.3096 - val_accuracy: 0.8306
Epoch 6/10
29/29 [==============================] - 112s 4s/step - loss: 0.3111 - accuracy: 0.8283 - val_loss: 0.3067 - val_accuracy: 0.8285
Epoch 7/10
29/29 [==============================] - 112s 4s/step - loss: 0.3079 - accuracy: 0.8289 - val_loss: 0.3036 - val_accuracy: 0.8324
Epoch 8/10
29

In [60]:
#trainer_model.save_weights('template_attention.h5')
#trainer_model.load_weights('template_attention.h5')

In [61]:
#generator model
gen_encoder = Model([encoder_input],[encoder_output3,encoder_h3,encoder_c3])

gen_decoder_values_input = Input(shape=(max_words_sbt,hidden_size))
gen_decoder_h_input = Input(shape=[hidden_size])
gen_decoder_c_input = Input(shape=[hidden_size])

gen_decoder_embedded = decoder_embedding(decoder_input)
gen_decoder_output,gen_decoder_h,gen_decoder_c = decoder_lstm(gen_decoder_embedded,initial_state=[gen_decoder_h_input,gen_decoder_c_input])

attn_context = attn_layer([gen_decoder_output,gen_decoder_values_input])
gen_decoder_output = Concatenate(axis=-1)([gen_decoder_output,attn_context])

gen_decoder_output = tanh_dense(gen_decoder_output)
gen_decoder_output = softmax_dense(gen_decoder_output)

gen_decoder = Model([decoder_input]+[gen_decoder_values_input,gen_decoder_h_input,gen_decoder_c_input],[gen_decoder_output,gen_decoder_h,gen_decoder_c])

# Test

In [62]:
def generate_from_encoder_input(encoder_input):
    encoder_input = encoder_input.reshape(1,-1)
    values,h,c = gen_encoder.predict(encoder_input)
    
    single_tok = np.zeros((1,1))
    single_tok[0,0] = word_to_ind_bdd['sostoken ']
    decoder_input = single_tok
    
    generated = []
    count = 0
    while(True):
        decoder_output,new_h,new_c = gen_decoder.predict([decoder_input]+[values,h,c])
        count +=1
        
        sampled_index = np.argmax(decoder_output[0,-1,:])
        sampled_word = ind_to_word_bdd[sampled_index]
        
        if sampled_word != ' eostoken' and sampled_index != 0:
            generated.append(sampled_word)
        if count >= max_words_bdd or sampled_word == ' eostoken':
            break
        
        h,c = new_h,new_c
        decoder_input[0,0] = sampled_index
    
    generated = ' '.join(generated)
    return generated

In [63]:

for i in range(520,525):
    print("\n<<sample encoder input SBT >>")
    print(decode(train_encoder_input_encoded[i],ind_to_word_sbt))
    print("\n")
    print("<<generated BDD >>")
    print(generate_from_encoder_input(train_encoder_input_encoded[i]))
    print("\n")
    print("<<Expected BDD>>")
    print(decode(train_decoder_label_encoded[i],ind_to_word_bdd))
    print("========================================\n")



<<sample encoder input SBT >>
 ( MethodDeclaration ( Modifier public  ) Modifier public  ( Modifier static  ) Modifier static  ( SimpleName TG2 ) SimpleName TG2 ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName TG0 ) SimpleName TG0 ) Parameter ( Parameter ( PrimitiveType int ) PrimitiveType int ( SimpleName TG3 ) SimpleName TG3 ) Parameter ( PrimitiveType boolean ) PrimitiveType boolean ( BlockStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName TG5 ) SimpleName TG5 ( UnaryExpr ( IntegerLiteralExpr 1 ) IntegerLiteralExpr 1 ) UnaryExpr ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarator ( PrimitiveType int ) PrimitiveType int ( SimpleName TG1 ) SimpleName TG1 ( IntegerLiteralExpr 9 ) IntegerLiteralExpr 9 ) VariableDeclarator ) VariableDeclarationExpr ) ExpressionStmt ( ExpressionStmt ( VariableDeclarationExpr ( VariableDeclarato

In [64]:
input_encoded = test_encoder_input_encoded[0]

In [65]:
def return_generated_template(input_encoded):
    generated = generate_from_encoder_input(input_encoded)
    generated = generated.split(' ')
    return generated

In [66]:
score = trainer_model.evaluate([train_encoder_input_encoded,train_decoder_input_encoded], train_decoder_label_encoded, verbose=0)
print("%s: %.2f%%" % (trainer_model.metrics_names[1], score[1]*100))

accuracy: 84.66%
